In [ ]:
from __future__ import absolute_import # need it for metaseq import in py27
import os
from IPython.display import IFrame
import pandas as pd
import sys
import numpy as np
sys.path.insert(0, '../JKBio/')
import ChIPHelper
import itertools
import igv
import pysam
import Datanalytics as da
import Helper
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import matplotlib
import bokeh
from bokeh.io import output_notebook
import multiprocessing
processes = multiprocessing.cpu_count()
output_notebook()
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipyparallel as ipp
#rc = ipp.Client()
#with rc[:].sync_imports():
#    import numpy
#    import pandas as pd
#    import pysam
#    import sys
#%px sys.path.insert(0, '../JKBio/')
#with rc[:].sync_imports():
   # import ChIPHelper

### Let's build the indexes with bowtie2  

In [ ]:
! bowtie2-build data/reference/reference.fa data/reference/index --threads 8

## adding the data bucket to path

In [ ]:
! gcsfuse --only-dir Chip_AML jkobject data/seqs

## extracting which one is single end and paired end

### Renaming ill formatted files

In [ ]:
! mv mp183-MV411-DMSO-H3K27ac-r1.fastq.gz mp183-MV411-DMSO_H3K27ac-r1.fastq.gz
! mv mp718-MV411-IRF2BP2-r9_R1_001.fastq.gz mp718-MV411-IRF2BP2-r9_R1.fastq.gz
! mv mp717-MV411-GATA2-r4_R1_001.fastq.gz mp717-MV411-GATA2-r4_R1.fastq.gz
! mv mp716-MV411-GSE1-r9_R1_001.fastq.gz mp716-MV411-GSE1-r9_R1.fastq.gz
! mv mp715-MV411-ZEB2-r8_R1_001.fastq.gz mp715-MV411-ZEB2-r8_R1.fastq.gz
! mv mp714-MV411-ZEB2-r7_R1_001.fastq.gz mp714-MV411-ZEB2-r7_R1.fastq.gz
! mv mp702-MV411_DMSO_8h-H3K27ac-r2.fastq.gz mp702-MV411-DMSO_8h_H3K27ac-r2.fastq.gz
! mv mp690-MV411_DMSO_8h-MAX-r1.fastq.gz mp690-MV411-DMSO_8h_MAX-r1.fastq.gz

In [ ]:
singlend, paired_end, pairedendict = ChIPHelper.extractPairedSingleEndFrom(folder='data/seqs/')

In [ ]:
data = {'group': [], 'replicate': [], 'fastq_1': [], 'fastq_2': [], 'antibody': [], 'control': []}
for rfile in singlend:
# should end with r2
    dfile = rfile.split('.')[0].split('-')
    data['group'].append(dfile[2])
    data['antibody'].append('')
    data['control'].append('INPUT')
    data['replicate'].append(data['group'].count(dfile[2]))
    data['fastq_1'].append('data/seqs/' + rfile)
    data['fastq_2'].append('')

In [ ]:
pd.DataFrame(data, columns=['group', 'replicate', 'fastq_1', 'fastq_2', 'antibody', 'control']).to_csv('singleend.csv',index=False)

In [ ]:
controlfile=True
control = 'INPUT'
path_to_fastq = 'data/seqs/'

In [ ]:
data = {'group': [], 'replicate': [], 'fastq_1': [], 'fastq_2': [], 'antibody': [], 'control': []}
for file1, file2 in Helper.grouped(paired_end, 2):
  # should end with r2
    dfile = file1.split('.')[0].split('-')
    data['group'].append(dfile[2])
    data['antibody'].append('')
    data['control'].append(control if control != dfile[2] else '')
    data['replicate'].append(dfile[3].split('r')[1][0])
    data['fastq_1'].append(path_to_fastq + file1)
    data['fastq_2'].append(path_to_fastq + file2)
if not controlfile:
    raise('we need control file')
    pd.DataFrame(data, columns=['group','replicate','fastq_1','fastq_2','antibody','control']).to_csv('design_paired.csv', index=False)

In [ ]:
data = {'group': [], 'replicate': [], 'fastq_1': [], 'fastq_2': [], 'antibody': [], 'control': []}
for file1, file2 in Helper.grouped(paired_end, 2):
  # should end with r2
    dfile = file1.split('.')[0].split('-')
    data['group'].append(dfile[2])
    data['antibody'].append('')
    data['control'].append(control if control != dfile[2] else '')
    data['replicate'].append(dfile[3].split('r')[1][0])
    data['fastq_1'].append(path_to_fastq + file1)
    data['fastq_2'].append(path_to_fastq + file2)
if not controlfile:
    raise('we need control file')
    pd.DataFrame(data, columns=['group','replicate','fastq_1','fastq_2','antibody','control']).to_csv('design_paired.csv', index=False)

In [ ]:
ChIPHelper.executeNFcore('data/seqs/')

## Aligning and computing peaks 

We have witnessed some warnings on the data for :

- POLII-r1
- MEF2D-r1_R1
- ZMYND8-r2
- GATA2-r4_R1_001

and maybe more 

In [ ]:
#paired end
CPU = 8
ChIPHelper.computeSingleEnd(singlend,CPU = CPU)

In [ ]:
ChIPHelper.computePairedEnd(pairedendict,CPU = CPU)

In [ ]:
ChIPHelper.bigWigFrom('data/mapped/')

In [ ]:
! Rscript data/peaks/mp585-MV411-MYC-r3/NA_model.r
from wand.image import Image as WImage
img = WImage(filename='NA_model.pdf')
img

## Analyzing the peaks

In [ ]:
bindings = ChIPHelper.loadNarrowPeaks(path='data/peaks/')

In [ ]:
SEgenes = pd.read_csv('SEgenes.csv')
CTF = pd.read_csv('CTF.csv', header=None)[0].tolist()

In [ ]:
set(bindings.type.tolist())- set(CTF)

In [ ]:
CTF.extend(['GATA2','IKZF1','LYL1' ,'PU1','SMC1'])

In [ ]:
CTF = list(set(CTF) - (set(CTF) - set(bindings.type.tolist())))
CTF

In [ ]:
bindings = bindings.reset_index().drop('index',1)

In [ ]:
zones = ChIPHelper.mergePeaks(bindings, 1000, TFlist = CTF)

In [ ]:
zones.shape

In [ ]:
zones['score'] = zones['score'].values/ np.linalg.norm(zones['score'].values)
zones['foldchange'] =  zones['foldchange'].values / np.linalg.norm(zones['foldchange'].values)
for val in CTF:
    zones[val] = zones[val] / np.linalg.norm(zones[val])

In [ ]:
zones[zones.columns[:10]].to_csv('mergepeaks.narrowPeak', sep='\t', header=None, index=False)

In [ ]:
zones.to_csv('saved.csv')

In [ ]:
zones = pd.read_csv('saved.csv',index_col=0)

In [ ]:
zones

## Correlation Matrix

In [ ]:
Helper.plotCorrelationMatrix(zones[zones.columns[11:]].T, names=zones.columns.tolist()[11:])

In [ ]:
path = "data/mapped/"
zones = pd.read_csv('saved.csv',index_col=0)
MYC = pysam.AlignmentFile(path+"mp585-MV411-MYC-r3.bam", "rb")
RUNX1= pysam.AlignmentFile(path+"mp300-MV411-RUNX1-r3_R1.bam", "rb")
ZEB2= pysam.AlignmentFile(path+"mp320-MV411-ZEB2-r1_R1.bam", "rb")
SP1= pysam.AlignmentFile(path+"mp325-MV411-SP1-r2_R1.bam", "rb")
MEF2D= pysam.AlignmentFile(path+"mp324-MV411-MEF2D-r1_R1.bam", "rb")
df = zones.sort_values(by=["MYC"],axis=0, ascending=False).reset_index()

In [ ]:
df = ChIPHelper.loadNarrowPeaks("data/peaks/mp585-MV411-MYC-r3/NA_peaks.narrowPeak")
df = df.sort_values(by=["foldchange"],axis=0, ascending=False).reset_index()

In [ ]:
df.iloc[18000]

In [ ]:
MYC.

In [ ]:
group = 500
CPU = 8
#with Pool(CPU) as p:
MYC = ChIPHelper.computePeaksAt(df,MYC,2000,20000, window=1000)
MEF2D = ChIPHelper.computePeaksAt(df,MEF2D,2000,20000, window=1000)
SP1 = ChIPHelper.computePeaksAt(df,SP1,2000,20000, window=1000)
RUNX1 = ChIPHelper.computePeaksAt(df,RUNX1,2000,20000, window=1000)


In [ ]:
ZEB2 = ChIPHelper.computePeaksAt(df,ZEB2,2000,20000, window=1000)

# Peak Profile accross highest MYC peaks

In [ ]:
normMYC = MYC[np.argsort(MYC.sum(1))[::-1]]
normMYC = normMYC / np.max(normMYC,1)[:,None]
normMEF2D = MEF2D[np.argsort(MYC.sum(1))[::-1]]
normMEF2D = normMEF2D / np.max(normMEF2D,1)[:,None]
normSP1 = SP1[np.argsort(MYC.sum(1))[::-1]]
normSP1 = normSP1 / np.max(normSP1,1)[:,None]
normRUNX1 = RUNX1[np.argsort(MYC.sum(1))[::-1]]
normRUNX1 = normRUNX1 / np.max(normRUNX1,1)[:,None]


In [ ]:
normMYC = normMYC[np.argsort(normMYC.sum(1))[::-1]]

In [ ]:
MYC.shape

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize = (5,10))
plt.box(on=None)
plt.axis('off')
ax = []
TFs = {'MYC':normMYC,'RUNX1':normRUNX1,'SP1':normSP1,'MEF2D':normMEF2D}
i = 1
cmap = ['Blues','Greens','Purples','Oranges', 'Reds']
for key, tf in TFs.iteritems():
    ax = [fig.add_subplot(1,len(TFs) , i)]
    i+=1
    plt.box(on=None)
    ax[-1].set_title(key)
    ax[-1].axis('off')
    plt.imshow(tf, interpolation='nearest', aspect='auto', cmap=cmap[i-1],vmax=1)
plt.savefig("TFs_MYC")

In [ ]:
a = [0,1,1,2,3,4,3,0,0,0,0,0]
size = 5
np.convolve(a, np.ones((size,)) / size, mode='valid')

In [ ]:
data = zones[zones.columns[11:]]
data = np.log2(data.values+1)

In [ ]:
model = AgglomerativeClustering(n_clusters=2,linkage="average", affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(data.T)
ii = itertools.count(data.T.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]

In [ ]:
Helper.scatter(TSNE(2,5).fit_transform(data.T), labels=zones.columns[11:],colors=labels)

In [ ]:
Helper.plotCorrelationMatrix(zones[zones.columns[[11+i for i in labels.argsort()]]].T, names=zones.columns[[11+i for i in labels.argsort()]].tolist())

In [ ]:
b = igv.Browser({"genome": "hg38"})

In [ ]:
#b.load_track({
#    "type": "wig",
#    "format": "bigWig",
#    "name": "RUNX1",
#    "url": "data/mapped/mp300-MV411-RUNX1-r3_R1.bw",
#    "sourceType":"file",
#})
#b.load_track({
#    "name": "RUNX1peaks",
#    "url": "data/peaks/mp300-MV411-RUNX1-r3_R1/NA_peaks.narrowPeak",
#    "format": "narrowPeak",
#    "type": "annotation",
#    "indexed": False,
#    "displayMode": "COLLAPSED"
#})
#b.load_track({
#    "type": "wig",
#    "format": "bigWig",
#    "name": "PU1",
#    "url": "data/mapped/mp301-MV411-PU1-r2_R1.bw",
#    "sourceType":"file",
#})
#b.load_track({
#    "name": "PU1peaks",
#    "url": "data/peaks/mp301-MV411-PU1-r2_R1/NA_peaks.narrowPeak",
#    "format": "narrowPeak",
#    "type": "annotation",
#    "indexed": False,
#    "displayMode": "COLLAPSED"
#})
b.load_track({
    "type": "wig",
    "format": "bigWig",
    "name": "MYC",
    "url": "data/mapped/mp301-MV411-PU1-r2_R1.bw",
    "sourceType":"file",
})
b.load_track({
    "name": "MYCpeaks",
    "url": "data/peaks/mp585-MV411-MYC-r3/NA_peaks.narrowPeak",
    "format": "narrowPeak",
    "type": "annotation",
    "indexed": False,
    "displayMode": "COLLAPSED"
})
b.load_track({
    "name": "Mergedpeaks",
    "url": "mergepeaks.narrowPeak",
    "format": "narrowPeak",
    "type": "annotation",
    "indexed": False,
    "displayMode": "COLLAPSED"
})

In [ ]:
b.show()

### TO remove
- IgG: basically no binding
- DMSO: control substance (MYC?) what is non control?
- CTCF, 
- H3K27ac, 
- MAX: should be bound at all MYC peaks
- POLII: RNA polymerase 2. should be at all transcribed sites (and thus at all chip peaks)
- MED1: help POL2

In [ ]:
minval = np.argsort(np.count_nonzero(data,0))

## What coverage can we achieve with 2, 3 TFs

In [ ]:
maxv = 0
for i in range(len(minval)):
    for j in range(len(minval)):
        tot = data[:,j] + data[:,i]
        if float(np.count_nonzero(tot)) / data.shape[0] > maxv:
            keep = [i,j]
            maxv = float(np.count_nonzero(tot)) / data.shape[0]
print(maxv,zones.columns[11:][keep])

In [ ]:
maxv = 0
for i in range(len(minval)):
    for j in range(len(minval)):
        for k in range(len(minval)):
            tot = data[:,k] + data[:,j] + data[:,i]
            if float(np.count_nonzero(tot)) / data.shape[0] > maxv:
                keep = [i,j,k]
                maxv = float(np.count_nonzero(tot)) / data.shape[0]

In [ ]:
print(maxv,zones.columns[11:][keep])

## What is the best coverage we can achieve with 5 TFs

In [ ]:
for l in range(len(minval)):
    for m in range(len(minval)):
        tot = data[:,keep[0]] + data[:,keep[1]] + data[:,keep[2]] + data[:,l] + data[:,m]
        if float(np.count_nonzero(tot)) / data.shape[0] > maxv:
            keepadd = [l,m]
            maxv = float(np.count_nonzero(tot)) / data.shape[0]

In [ ]:
print(maxv,zones.columns[11:][keepadd])

## Sum of the X most bound TFs

In [ ]:
float(np.count_nonzero(np.sum(data[:,minval[-3:]],1))) / data.shape[0]

## Most globally bound TFs

In [ ]:
zones.columns[11:][minval[-6:]]

## Recap
here we have confirmed that POL2 is present at all merged peaks. and that IgG is only present at 0.4% of mergedpeaks. we confirmed that MED1 is present in almost all POL2 merged peaks
why IgG and POL2 are very correlated?? should not be the case..


CEBPA covers 96% of all tfs, 97% are covered by adding MEF2D and ZEB2. 98.5% by adding IKZF1 and GSE1

CEBPA, FLI1, SMC1, MEIS1, IRF8, PU1 together summarize 99.97% of the merged peaks